# Do common SQL tasks in Python

### All the required libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import math

### Getting the HTML page and returning a BeautifulSoup object

In [2]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita')
gdpTable = r.text
soup = BeautifulSoup(gdpTable, 'lxml')


### Parsing the table 

In [3]:
table = soup.find('table', attrs = {"class" :"wikitable sortable"})
theads=[]
for tx in table.findAll('th'):
    theads.append(tx.text)

data =[]
for rows in table.findAll('tr'):
        row={}
        i=0
        for cell in rows.findAll('td'):
            row[theads[i]]=re.sub('\xa0', '',cell.text)
            i+=1
        if len(row)!=0:
            data.append(row)


### Making a dataframe using the dict obtained by parsing the table

In [4]:
df = pd.DataFrame(data)

In [5]:
df.head()

,Country,Rank,US$
0,Luxembourg,1,"103,199"
1,Switzerland,2,"79,242"
2,Norway,3,"70,392"
3,Macau,—,"67,079"
4,Ireland,4,"62,562"


### Renaming the columns  since it is easier to manipulate if the special symbol is removed

In [6]:
df = df.rename(columns = {"US$" : "gdp_in_usd"})

In [7]:
df.head()

,Country,Rank,gdp_in_usd
0,Luxembourg,1,"103,199"
1,Switzerland,2,"79,242"
2,Norway,3,"70,392"
3,Macau,—,"67,079"
4,Ireland,4,"62,562"


### Removing the rank column since the data is corrupted


In [8]:
del df['Rank']

In [9]:
df.head()

,Country,gdp_in_usd
0,Luxembourg,"103,199"
1,Switzerland,"79,242"
2,Norway,"70,392"
3,Macau,"67,079"
4,Ireland,"62,562"


### Removing commas in the values and making them 'int' type

In [10]:
df['gdp_in_usd'] = df['gdp_in_usd'].apply(lambda x: re.sub(',', '', x))

In [11]:
df['gdp_in_usd'] = df['gdp_in_usd'].apply(pd.to_numeric)

In [12]:
df.head()

,Country,gdp_in_usd
0,Luxembourg,103199
1,Switzerland,79242
2,Norway,70392
3,Macau,67079
4,Ireland,62562


### Filtering out all the countries  by the starting character of their name

In [19]:
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWYZ"              #Does not contain X, since no country's name starts with X
df_by_initial = [None]*26
count = 0
for x in alphabets:
     df_by_initial[count] = df[df['Country'].str.startswith(x)]
     count += 1

### Finding the mean GDP of t the countries with the same intial character

In [21]:
means = {}
count = 0
for x in alphabets:
    unrounded = df_by_initial[count].gdp_in_usd.mean() 
    means[x] = math.ceil(unrounded*100)/100
    count +=1

{'A': 14402.0,
 'B': 10512.0,
 'C': 8618.36,
 'D': 17541.75,
 'E': 6185.63,
 'F': 28826.34,
 'G': 8415.73,
 'H': 14919.0,
 'I': 25575.13,
 'J': 16467.34,
 'K': 7496.8,
 'L': 19028.25,
 'M': 8453.16,
 'N': 19105.45,
 'O': 15964.0,
 'P': 10880.3,
 'Q': 60787.0,
 'R': 6374.34,
 'S': 16959.25,
 'T': 6742.0,
 'U': 22263.29,
 'V': 4748.67,
 'W': 10038.0,
 'Y': 938.0,
 'Z': 1126.0}